In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch.nn as nn
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time, torch
from tqdm import tqdm
warnings.filterwarnings('ignore')
import os
import random
import torch.optim as optim
import torchvision.transforms as transforms, datasets
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from tqdm import tqdm
root = "/content/data/"

In [29]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.8.0+cu126
CUDA available: True


In [30]:
SHARED_DATA_DIR =  "/content/drive/MyDrive/Olympic AI/CV"
LOCAL_DATA_DIR = "/content/data"

!mkdir -p "/content/data" "$LOCAL_DATA_DIR"

!rsync -ah --progress "$SHARED_DATA_DIR"/ "$LOCAL_DATA_DIR"/



sending incremental file list


In [ ]:
# 1. Import thư viện
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm # Để hiển thị tiến độ
from sklearn.metrics import f1_score # Để tính toán F1-score
import time # Để đo thời gian

# --- Đường dẫn thư mục gốc của bạn ---
# Vui lòng điều chỉnh biến 'root' này để trỏ đến thư mục chứa 'dataset' của bạn.
# Ví dụ: nếu cấu trúc là /path/to/your/project/dataset/train/images,
# thì root = '/path/to/your/project'

# 2. Thiết lập Seed để tái lập kết quả
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Cài đặt này có thể làm chậm quá trình huấn luyện nhưng đảm bảo tái lập kết quả trên GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Đã thiết lập seed = {seed}")

SEED = 42
set_seed(SEED)

# 3. Định nghĩa Transform cho ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet18 thường nhận đầu vào 224x224
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # Mean và Std chuẩn của ImageNet
        std=[0.229, 0.224, 0.225]
    ),
])
print("Đã định nghĩa transform")

# 4. Xây dựng Dataset Class
class StormDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.img_dir = os.path.join(root_dir, "images")
        self.transform = transform
        self.is_test = is_test

        if is_test:
            # Đối với tập test, không có nhãn. annotations.csv chỉ chứa file_name.
            # Hoặc nếu không có annotations.csv, chỉ cần đọc danh sách ảnh.
            # Để an toàn, chúng ta sẽ tạo một DataFrame từ các tên file ảnh.
            self.image_files = sorted([f for f in os.listdir(self.img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            self.annotations = pd.DataFrame({'file_name': self.image_files})
        else:
            # Đối với tập train/val, cần file annotations.csv có nhãn.
            annotations_path = os.path.join(root_dir, "annotations.csv")
            if not os.path.exists(annotations_path):
                raise FileNotFoundError(f"File annotations.csv không tìm thấy tại: {annotations_path}")
            self.annotations = pd.read_csv(annotations_path)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        row = self.annotations.iloc[idx]
        img_path = os.path.join(self.img_dir, row['file_name'])

        # Kiểm tra xem file ảnh có tồn tại không
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File ảnh không tìm thấy tại: {img_path}")

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # Logic nhãn:
        # Nếu là tập test, nhãn mặc định là -1 (hoặc một giá trị không hợp lệ khác)
        # hoặc có thể bỏ qua trả về nhãn. Ở đây giữ -1 để tương thích với pipeline.
        if self.is_test:
            # Dù có cột 'is_negative' hay 'category_id' trong file annotations.csv của test set,
            # chúng ta không sử dụng chúng để huấn luyện.
            # Giá trị trả về nhãn sẽ là -1 để báo hiệu đây là tập test.
            return image, -1
        else:
            # Đối với tập train/val, đọc nhãn từ annotations.csv
            # is_negative: True -> label 0
            # category_id: 1, 2, 3, 4 -> label 1, 2, 3, 4
            # Tổng cộng có 5 lớp (0, 1, 2, 3, 4)
            if row['is_negative']:
                label = 0
            else:
                label = int(row['category_id'])
            return image, label

print("Đã định nghĩa StormDataset")


# 5. Xây dựng Model CNN (ResNet18) - Thay thế SimpleCNN
# ------------------------------------------------------------------------------------
# Định nghĩa BasicBlock cho ResNet18/34
class BasicBlock(nn.Module):
    """
    Khối xây dựng cơ bản của ResNet cho các phiên bản như ResNet18 và ResNet34.
    Bao gồm hai lớp tích chập 3x3 và một kết nối tắt (shortcut connection).
    """
    expansion = 1 # Hệ số mở rộng số kênh đầu ra. Với BasicBlock, số kênh không đổi.

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        # Lớp tích chập đầu tiên trong block
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        # Lớp tích chập thứ hai trong block
        self.conv2 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels * self.expansion)

        # Kết nối tắt (shortcut connection)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            # Nếu kích thước không gian (stride != 1) hoặc số kênh thay đổi,
            # cần một phép tích chập 1x1 trên nhánh tắt để khớp kích thước.
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        identity = x # Lưu trữ đầu vào cho kết nối tắt

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity) # Cộng nhánh chính với nhánh tắt (residual connection)
        out = self.relu(out)
        return out

# Định nghĩa kiến trúc ResNet tổng quát
class ResNet(nn.Module):
    """
    Kiến trúc ResNet tổng quát.
    Sử dụng các BasicBlock (hoặc BottleneckBlock cho các phiên bản sâu hơn)
    để xây dựng mô hình.
    """
    def __init__(self, block, num_blocks, num_classes=5, in_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64 # Số kênh đầu ra sau lớp conv1

        # Lớp tích chập khởi tạo: Thường là kernel_size=7, stride=2, MaxPool
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Các tầng ResNet (layer stages)
        # Mỗi tầng chứa một số lượng block nhất định, và tầng đầu tiên của mỗi nhóm (trừ layer1)
        # sẽ thực hiện downsampling (stride=2)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Lớp Global Average Pooling để giảm kích thước không gian xuống 1x1
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Lớp Fully Connected cuối cùng để phân loại
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # Khởi tạo trọng số (quan trọng khi train từ scratch)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, layer_block, out_channels, num_blocks, stride):
        """
        Hàm trợ giúp để tạo một tầng (stage) của ResNet.
        Trong một tầng, block đầu tiên có thể có stride > 1 để downsample.
        """
        strides = [stride] + [1] * (num_blocks - 1) # stride chỉ áp dụng cho block đầu tiên
        layers = []
        for s in strides:
            layers.append(layer_block(self.in_channels, out_channels, s))
            self.in_channels = out_channels * layer_block.expansion # Cập nhật in_channels cho block tiếp theo
        return nn.Sequential(*layers)

    def forward(self, x):
        # Forward pass qua lớp khởi tạo
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Forward pass qua các tầng ResNet
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Forward pass qua Global Average Pooling và lớp Fully Connected
        x = self.avgpool(x)
        x = torch.flatten(x, 1) # Làm phẳng đầu ra thành vector
        x = self.fc(x)
        return x

# Hàm tạo ResNet18 cụ thể
def ResNet18(num_classes=5, in_channels=3):
    """
    Hàm khởi tạo mô hình ResNet18.
    Sử dụng BasicBlock và cấu hình số lượng block cho mỗi tầng: [2, 2, 2, 2].
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes, in_channels)

# ------------------------------------------------------------------------------------

# Khởi tạo ResNet18
# num_classes = 5 (0, 1, 2, 3, 4) vì có cả lớp 'is_negative' (0) và 4 cấp độ bão (1-4)
model = ResNet18(num_classes=5, in_channels=3)
print(f"Đã xây dựng mô hình ResNet18")
print(f"Tổng số parameters: {sum(p.numel() for p in model.parameters()):,}")


# 6. Chuẩn bị DataLoader
def worker_init_fn(worker_id):
    """Đảm bảo tái lập kết quả cho các worker của DataLoader."""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

full_dataset = StormDataset(os.path.join(root, "dataset/train"), transform=transform)
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

print(f"Tổng số ảnh huấn luyện/validation: {total_size}")
print(f"Tập huấn luyện: {train_size}, Tập validation: {val_size}")

generator = torch.Generator().manual_seed(SEED) # Dùng cho random_split và DataLoader workers
train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size], generator=generator
)

BATCH_SIZE = 64 # Có thể điều chỉnh Batch Size
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    worker_init_fn=worker_init_fn, generator=generator, # generator cho shuffle và worker_init_fn
    num_workers=4, # Tăng num_workers nếu CPU/RAM cho phép để tăng tốc load dữ liệu
    pin_memory=True # Giúp tăng tốc độ chuyển dữ liệu lên GPU
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    worker_init_fn=worker_init_fn, # Đảm bảo tái lập kết quả
    num_workers=4,
    pin_memory=True
)
print(f"Đã tạo DataLoader (batch_size={BATCH_SIZE})")


# 7. Train Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Sử dụng device: {device}")

model.to(device) # Chuyển mô hình sang thiết bị tính toán

# Criterion (Hàm mất mát) và Optimizer
criterion = nn.CrossEntropyLoss()
# Sử dụng AdamW thay vì Adam thường để có khả năng điều chỉnh Weight Decay tốt hơn
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4) # Giảm LR và thêm Weight Decay

# Learning Rate Scheduler (giúp cải thiện hiệu suất, đặc biệt với các mạng sâu)
# Giảm learning rate khi validation loss ngừng cải thiện
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

NUM_EPOCHS = 25 # Tăng số epoch để ResNet18 có đủ thời gian học từ scratch
print(f"Bắt đầu huấn luyện ({NUM_EPOCHS} epochs)...")

history = {
    'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': []
}
best_val_f1 = -1.0
best_epoch = -1

for epoch in range(NUM_EPOCHS):
    start_epoch_time = time.time()

    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = 100 * correct / total

    # --- Validation Phase ---
    model.eval()
    val_correct, val_total = 0, 0
    val_running_loss = 0.0
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)

            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(predicted.cpu().numpy())

    val_loss = val_running_loss / val_total
    val_acc = 100 * val_correct / val_total

    # Tính F1-score trên tập validation
    # average='weighted' là lựa chọn tốt cho các tập dữ liệu không cân bằng
    val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')

    # Cập nhật Learning Rate Scheduler
    scheduler.step(val_loss)

    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)

    epoch_duration = time.time() - start_epoch_time

    print(f"\nEpoch [{epoch+1}/{NUM_EPOCHS}] ({epoch_duration:.2f}s):")
    print(f" Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f" Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}")

    # Lưu model tốt nhất dựa trên Val F1-score
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_epoch = epoch + 1
        model_path = os.path.join(root, "resnet18_best_f1_model.pth")
        torch.save(model.state_dict(), model_path)
        print(f"  >>> Lưu model tốt nhất với Val F1: {best_val_f1:.4f} tại Epoch {best_epoch} <<<")

print("\nHoàn thành huấn luyện!")
print(f"Model tốt nhất đạt Val F1 = {best_val_f1:.4f} tại Epoch {best_epoch}")

# 8. Lưu Model cuối cùng và lịch sử huấn luyện
# (Chỉ lưu model tốt nhất trong vòng lặp)
# model_path = os.path.join(root, "resnet18_final_model.pth")
# torch.save(model.state_dict(), model_path)
# print(f"Đã lưu model cuối cùng tại: {model_path}")

history_path = os.path.join(root, "training_history_resnet18.csv")
history_df = pd.DataFrame(history)
history_df.to_csv(history_path, index=False)
print(f"Đã lưu lịch sử huấn luyện tại: {history_path}")


# --- Tải lại model tốt nhất để suy luận ---
print("\nĐang tải lại model tốt nhất để suy luận...")
model.load_state_dict(torch.load(model_path))
model.to(device)
print(f"Đã tải model từ '{model_path}'")


# 9. Inference trên Public Test Set
print("\nBắt đầu inference trên PUBLIC TEST SET...")
public_test_dataset = StormDataset(os.path.join(root, "dataset/public_test"), transform=transform, is_test=True)
public_test_loader = DataLoader(
    public_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True
)

model.eval() # Chuyển model sang chế độ đánh giá
file_names = []
preds_is_negative = []
preds_category_id = []

with torch.no_grad(): # Không tính toán gradient trong quá trình suy luận
    for i, (images, _) in enumerate(tqdm(public_test_loader, desc="Public Test")):
        images = images.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1) # Lấy lớp có điểm số cao nhất

        # Lấy tên file tương ứng từ DataFrame của Dataset
        batch_start = i * BATCH_SIZE
        batch_end = min(batch_start + images.size(0), len(public_test_dataset.annotations)) # Đảm bảo không vượt quá kích thước dataset
        batch_files = public_test_dataset.annotations.iloc[batch_start:batch_end]['file_name'].values
        file_names.extend(batch_files)

        for p in predicted.cpu().numpy():
            if p == 0: # Lớp 0 tương ứng với is_negative = True
                preds_is_negative.append(True)
                preds_category_id.append("") # category_id để trống nếu is_negative là True
            else: # Các lớp 1-4 tương ứng với category_id
                preds_is_negative.append(False)
                preds_category_id.append(int(p)) # Chuyển về int để lưu

df_public = pd.DataFrame({
    'file_name': file_names,
    'is_negative': preds_is_negative,
    'category_id': preds_category_id
})

public_csv_path = os.path.join(root, "public_cv.csv")
df_public.to_csv(public_csv_path, index=False)
print(f"Đã tạo file public_cv.csv tại {public_csv_path} với {len(df_public)} dự đoán!")
print("\nPreview kết quả Public Test:")
print(df_public.head(10))


# 10. Inference trên Private Test Set
print("\nBắt đầu inference trên PRIVATE TEST SET...")
private_test_dataset = StormDataset(os.path.join(root, "dataset/private_test"), transform=transform, is_test=True)
private_test_loader = DataLoader(
    private_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True
)

model.eval() # Chuyển model sang chế độ đánh giá
file_names = []
preds_is_negative = []
preds_category_id = []

with torch.no_grad(): # Không tính toán gradient trong quá trình suy luận
    for i, (images, _) in enumerate(tqdm(private_test_loader, desc="Private Test")):
        images = images.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1) # Lấy lớp có điểm số cao nhất

        # Lấy tên file tương ứng từ DataFrame của Dataset
        batch_start = i * BATCH_SIZE
        batch_end = min(batch_start + images.size(0), len(private_test_dataset.annotations)) # Đảm bảo không vượt quá kích thước dataset
        batch_files = private_test_dataset.annotations.iloc[batch_start:batch_end]['file_name'].values
        file_names.extend(batch_files)

        for p in predicted.cpu().numpy():
            if p == 0: # Lớp 0 tương ứng với is_negative = True
                preds_is_negative.append(True)
                preds_category_id.append("") # category_id để trống nếu is_negative là True
            else: # Các lớp 1-4 tương ứng với category_id
                preds_is_negative.append(False)
                preds_category_id.append(int(p)) # Chuyển về int để lưu

df_private = pd.DataFrame({
    'file_name': file_names,
    'is_negative': preds_is_negative,
    'category_id': preds_category_id
})

private_csv_path = os.path.join(root, "private_cv.csv")
df_private.to_csv(private_csv_path, index=False)
print(f"Đã tạo file private_cv.csv với {len(df_private)} dự đoán!")
print("\nPreview kết quả Private Test:")
print(df_private.head(10))


Đã thiết lập seed = 42
Đã định nghĩa transform
Đã định nghĩa StormDataset
Đã xây dựng mô hình ResNet18
Tổng số parameters: 11,179,077
Tổng số ảnh huấn luyện/validation: 8385
Tập huấn luyện: 6708, Tập validation: 1677
Đã tạo DataLoader (batch_size=64)
🔧 Sử dụng device: cuda
Bắt đầu huấn luyện (25 epochs)...


Epoch 1/25 [Train]:  57%|█████▋    | 60/105 [00:44<00:22,  2.02it/s]